# Fornax Cloud Access Demo

This notebook illustrates the cloud access tools developed as part of the Fornax project.

It uses `pyvo`, a python client that uses the Virtual Observatory protocols to find data, then it uses the code from the `fornax` library to download data from the cloud. These features are illustrated by finding data for the galaxy M82 from HST, Galex, Chandra and Spitzer and downloading it from AWS.


**Author**: Abdu Zoghbi (HEASARC); on behalf of the Fornax Team (HEASARC, MAST, IRSA)

**Date**: 10-Jul-2023

### Import relevant Modules

In [ ]:
import os
import sys

sys.path.insert(0, os.getcwd())
from demo_helpers import *
import fornax


### Define some sky position

In [ ]:
pos = coord.SkyCoord.from_name("M82")

---
## 1. Search for HST Data for the Source

In [ ]:
hst_url = 'https://mast.stsci.edu/portal_vo/Mashup/VoQuery.asmx/SiaV1?MISSION=HST&'
hst_service = pyvo.dal.sia.SIAService(hst_url)
query_result = hst_service.search(pos=(pos.ra.deg, pos.dec.deg), size=10*u.arcsec)
print(f'The query returned {len(query_result)} entries')

In [ ]:
hst_result = filter_hst_results(query_result)
print(f'Filtering resulted in {len(hst_result)} entries')

### 1.1. Download data from On-prem servers:

In [ ]:
hst_product = hst_result[-4]
hst_prem = fornax.get_data_product2(hst_product, 'prem')
hst_file = hst_prem.download()
print(f'\nThe Downloaded file is: {hst_file}')

### 1.2. Download data from AWS:
First we show how the cloud information is served.

In [ ]:
hst_product['cloud_access']

Then, we download it from the cloud, and plot it

In [ ]:
# hst_aws = fornax.get_data_product2(hst_product, 'aws')
# hst_file = hst_aws.download()

In [ ]:
fig = plt.figure(figsize=(10,10))
plt.imshow(plt.imread(hst_file))

---
## 2. Search for Galex Data for the Source

In [ ]:
# Query data provider 
galex_url = hst_url.replace('HST', 'GALEX')
galex_service = pyvo.dal.sia.SIAService(galex_url)
query_result = galex_service.search(pos=(pos.ra.deg, pos.dec.deg), size=10*u.arcsec)
print(f'The query returned {len(query_result)} entries')

In [ ]:
# filter GALEX data
galex_result = filter_galex_results(query_result)
print(f'Filtering resulted in {len(galex_result)} entries')

### 2.1 Download Galex Data from Cloud & Plot it

In [ ]:
#galex_prem = fornax.get_data_product2(galex_result[0], 'prem')
#galex_file = galex_prem.download()

# again, we print the cloud information provided
print(galex_result[0]['cloud_access'])

galex_aws = fornax.get_data_product2(galex_result[1], 'aws')
galex_file = galex_aws.download()

print(f'\nThe Downloaded file is: {galex_file}')

In [ ]:
fig = plt.figure(figsize=(10,10))
plt.imshow(plt.imread(galex_file)[600:-600,600:-600])

---
## 3. Search for Chandra Data for the Source

In [ ]:
# Query data provider 
chandra_url = 'https://heasarc.gsfc.nasa.gov/xamin_aws/vo/sia?table=chanmaster'
chandra_service = pyvo.dal.sia.SIAService(chandra_url)
query_result = chandra_service.search(pos=(pos.ra.deg, pos.dec.deg), size=10*u.arcsec)

print(f'The query returned {len(query_result)} entries')

In [ ]:
#chandra_prem = fornax.get_data_product2(chandra_result[-1], 'prem')
#chandra_file = chandra_prem.download()

# again, we print the cloud information provided
print(query_result[-1]['cloud_access'])

chandra_aws = fornax.get_data_product2(query_result[-5], 'aws', 'ucd')
chandra_file = chandra_aws.download()

print(f'\nThe Downloaded file is: {chandra_file}')

In [ ]:
fig = plt.figure(figsize=(10,10))
plt.imshow(plt.imread(chandra_file)[600:-50,300:-300])

---
## 4. Search for Spitzer Data for the Source

In [ ]:
# Query data provider 
spizter_url = 'https://irsa.ipac.caltech.edu/SIA'
spizter_service = pyvo.dal.sia2.SIAService(spizter_url)
query_result = spizter_service.search(pos=(pos.ra.deg, pos.dec.deg, 10*u.arcsec),
                                      collection='spitzer_seip')
# get it in a table format
spitzer_result = query_result.to_table()
spitzer_result.sort('access_url')


print(f'The query returned {len(query_result)} entries')

### 4.1 Download Spitzer Data from Cloud & Plot a cutout

We time it to check performance

In [ ]:
t0 = time.time()
#spitzer_prem = fornax.get_data_product2(spitzer_result[4], 'prem')
#spitzer_file = spitzer_prem.download()

# again, we print the cloud information provided
print(spitzer_result[4]['cloud_access'])

spitzer_aws = fornax.get_data_product2(spitzer_result[4], 'aws')
spitzer_file = spitzer_aws.download()

print(f'\nThe Downloaded file is: {spitzer_file}')
print(f'It took {time.time() - t0:.3} seconds')

In [ ]:
fig = plt.figure(figsize=(10,10))
with fits.open(spitzer_file) as fp:
    plt.imshow(fp[0].data[2500:3360,1800:2730], norm='symlog')

### 4.2 Do Cloud Cutout without Downloading the data

In [ ]:
# using cloud cutout
s3_uri = spitzer_aws.get_links()[0]
print(s3_uri)

In [ ]:
t0 = time.time()
with fits.open(s3_uri, fsspec_kwargs={"anon": True}) as hdul:  
    cutout = hdul[0].section[2500:3360,1800:2730]
print(f'Cutout time {time.time() - t0:.3} seconds')

In [ ]:
fig = plt.figure(figsize=(10,10))
plt.imshow(cutout, norm='symlog')

---
## 5. Put all the images together

In [ ]:
fig,axs = plt.subplots(1,4,figsize=(14,4))

axs[0].imshow(plt.imread(hst_file))
axs[0].set_title('HST')

axs[1].imshow(plt.imread(galex_file)[800:-800,800:-800])
axs[1].set_title('Galex')

axs[2].imshow(plt.imread(chandra_file)[600:-50,300:-300])
axs[2].set_title('Chandra')

axs[3].imshow(cutout, norm='symlog')
axs[3].set_title('Spitzer')